In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Float, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
import pandas as pd
import os

Base = declarative_base()

# Membaca data dari .env
from dotenv import load_dotenv
load_dotenv()

# Koneksi database menggunakan .env
db_username = os.getenv('DB_USERNAME')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

db_url = f"postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(db_url)
Session = sessionmaker(bind=engine)

class Customer(Base):
    __tablename__ = 'customers'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    email = Column(String, unique=True)
    phone = Column(String)
    address = Column(String)
    created_at = Column(DateTime)

class Product(Base):
    __tablename__ = 'products'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    category = Column(String)
    price = Column(Float)
    stock = Column(Integer)

class Order(Base):
    __tablename__ = 'orders'

    id = Column(Integer, primary_key=True)
    customer_id = Column(Integer, ForeignKey('customers.id'))
    order_date = Column(DateTime)
    total_amount = Column(Float)

    customer = relationship("Customer", backref="orders")  # Relasi dengan tabel customers

class OrderItem(Base):
    __tablename__ = 'order_items'

    id = Column(Integer, primary_key=True)
    order_id = Column(Integer, ForeignKey('orders.id'))
    product_id = Column(Integer, ForeignKey('products.id'))
    quantity = Column(Integer)
    price = Column(Float)

    order = relationship("Order", backref="order_items")  # Relasi dengan tabel orders
    product = relationship("Product", backref="order_items")  # Relasi dengan tabel products

class Shipment(Base):
    __tablename__ = 'shipments'

    id = Column(Integer, primary_key=True)
    order_id = Column(Integer, ForeignKey('orders.id'))
    shipment_date = Column(DateTime)
    status = Column(String)
    tracking_number = Column(String)
    delivery_date = Column(DateTime)

    order = relationship("Order", backref="shipments")  # Relasi dengan tabel orders

# Buat semua tabel di database
Base.metadata.create_all(engine)


In [ ]:
def insert_csv_to_postgresql(csv_file, table_name):
    try:
        # Membaca data dari CSV
        df = pd.read_csv(f"data/{csv_file}")

        # Menyisipkan data ke tabel menggunakan pandas to_sql
        df.to_sql(table_name, engine, if_exists='append', index=False)

        print(f"Data dari {csv_file} berhasil dimasukkan ke tabel {table_name}!")
    except Exception as e:
        print(f"Error saat memasukkan data dari {csv_file}: {e}")

# Masukkan data dari CSV
insert_csv_to_postgresql('customers.csv', 'customers')
insert_csv_to_postgresql('products.csv', 'products')
insert_csv_to_postgresql('orders.csv', 'orders')
insert_csv_to_postgresql('order_items.csv', 'order_items')
insert_csv_to_postgresql('shipments.csv', 'shipments')
